# Efficiency: loop interchange; threads
_COSC 208, Introduction to Computer Systems, 2023-11-15_

## Announcements
* Project 4 due Thursday, December 7

## Outline
* Warm-up
* Loop interchange
* Threads and memory

## Warm-up (Q3 & Q4 from last class)

* Assume a cache can hold 3 entries and the following 15 data accesses occur: 
```
3, 4, 4, 5, 3, 2, 3, 4, 1, 4, 4, 2, 5, 2, 4
```
* Q1: _What is the sequence of hits, insertions, and replacements that occur when a **first in first out (FIFO)** cache replacement algorithm is used?_

```
+3, +4, H4, +5, H3, -3/+2, -4/+3, -5/+4, -2/+1, H4, H4, -3/+2, -4/+5, H2, -1/+4
Hit ratio = 5/15 = 33%
```

<p style="height:6em;"></p>

* Q2: _What is the sequence of hits, insertions, and replacements that occur when a **least recently used (LRU)** cache replacement algorithm is used?_

```
+3, +4, H4, +5, H3, -4/+2, H3, -5/+4, -2/+1, H4, H4, -3/+2, -1/+5, H2, H4
Hit ratio = 7/15 = 47%
```

<p style="height:6em;"></p>

🛑 **STOP here** after completing the above question; if you have extra time please **skip ahead** to the extra practice.

## Loop interchange

* Example

In [16]:
#include <stdlib.h>
#include <stdio.h>
#define LEN 12
int main() {
    int *array = malloc(sizeof(int) * LEN);

    for (int i = 0; i < LEN; i++) {
        array[i] = i;
    }
    
    int sum = 0;
    for (int j = 0; j < 4; j++) {
        for (int k = 0; k < LEN; k += 4) {
            sum += array[j+k];
        }
    }
    printf("%d\n", sum);
}

66


* _Assume the values of all local variables are stored in registers (**not** the stack) and the value of `array` is `0x400`. What is the sequence of memory addresses that are accessed?_
    * First for loop: `0x400`, `0x404`, `0x408`, `0x40c`, `0x410`, `0x414`, `0x418`, `0x41c`, `0x420`, `0x424`, `0x428`, `0x42c`
    * Second for loop: `0x400`, `0x410`, `0x420`, `0x404`, `0x414`, `0x424`, `0x408`, `0x418`, `0x428`, `0x40c`, `0x41c`, `0x42c`, 
    * Notice that the first for loop accesses memory addresses in order, whereas the second for loop accesses addresses out of order
* _Now assume the system uses a cache that holds 2 entries which are each 16 bytes large. What is the sequence of hits and misses using a least recently used (LRU) replacement policy?_
    * First for loop: Miss (+0x4000), Hit, Hit, Hit, Miss (+0x4010), Hit, Hit, Hit, Miss (-0x4000/+0x4020), Hit, Hit, Hit
    * Second for loop: Miss (-0x4010/+0x4000), Miss (-0x4020/+0x4010), Miss (-0x4000/+0x4020), Miss (-0x4010/+0x4000), Miss (-0x4020/+0x4010), Miss (-0x4000/+0x4020), Miss (-0x4010/+0x4000), Miss (-0x4020/+0x4010), Miss (-0x4000/+0x4020), Miss (-0x4010/+0x4000), Miss (-0x4020/+0x4010), Miss (-0x4000/+0x4020)
    * Notice that the first for loop has three hits after each miss, whereas the second for loop is all misses
* _How could we modify the code to achieve a higher hit ratio?_ – loop interchange, i.e., swap inner and outer loops

In [17]:
#include <stdlib.h>
#include <stdio.h>
#define LEN 12
int main() {
    int *array = malloc(sizeof(int) * LEN);

    for (int i = 0; i < LEN; i++) {
        array[i] = i;
    }
    
    int sum = 0;
    for (int k = 0; k < LEN; k += 4) {
        for (int j = 0; j < 4; j++) {
            sum += array[j+k];
        }
    }
    printf("%d\n", sum);
}

66


<div style="page-break-after:always;"></div>

* Q3: _Would loop interchange improve the efficiency of this code?_

In [ ]:
void hundreds() {
    int *nums = malloc(sizeof(int) * 1000);
    for (int i = 0; i < 1000; i+= 100) {
        for (int j = 0; j < 100; j++) {
            nums[i+j] = i;
        }
    }
}

    No

<p style="height:2em;"></p>

* Q4: _Would loop interchange improve the efficiency of this code?_

In [ ]:
void multiplication(int grid[][], int rows, int cols) {
    for (int c = 0; c < cols; c++) {
        for (int r = 0; r < rows; r++) {
            grid[r][c] = c * r;
        }
    }
}

    Yes

<p style="height:2em;"></p>

🛑 **STOP here** after completing the above question; if you have extra time please **skip ahead** to the extra practice.

## Threads and memory

* Threads are multiple execution contexts within the **same process**
    * Processes are multiple execution contexts within the **same machine**
* Because threads are within the same process, they share all of the process’s resources — memory, CPU time, file descriptors (i.e., open files), etc.
* Consequently, two threads can update the same variable

* Example

In [ ]:
#include <stdio.h>
#include <stdlib.h>
void *thread1_main(void *arg) {
    int *x = (int *)arg;
    *x += 1;
    return NULL;
}
void *thread2_main(void *arg) {
    int *y = (int *)arg;
    *y += 2;
    return NULL;
}
int main() {
    int *z = malloc(sizeof(int));
    *z = 0;
    // Create thread running thread1_main(z)
    // Create thread running thread2_main(z)
    // Wait for threads to finish
    printf("z is %d\n", *z);
}

```
z is 3
```

* Two processes cannot update the same variable — memory is not shared; must use inter-process communication mechanism to share information

<div style="page-break-after:always;"></div>

* Q5: _What are all possible outputs produced by this program?_

In [ ]:
#include <stdio.h>
#include <stdlib.h>
void *increment(void *arg) {
    int *num = (int*)arg;
    *num += 1;
    return NULL;
}
void *zero(void *arg) {
    int *num = (int*)arg;
    *num = 0;
    return NULL;
}
int main() {
    int *i = malloc(sizeof(int));
    *i = 5;
    // Create thread running increment(i)
    // Create thread running zero(i)
    // Wait for threads to finish
    printf("i=%d\n", *i);
}

```
i=0
```
OR
```
i=1
```

<p style="height:10em;"></p>

## Extra practice

Q6: _For each of the following scenarios, indicate whether it is an example of temporal locality, spatial locality, or neither._

* Gates for flights on the same airline are located in the same airport terminal/concourse – spatial locality
* A grocery list is arranged in alphabetical order – neither
* Clothes in a closet are grouped into outfits, with a shirt and a pair of pants stored next to each other – spatial locality
* Boxes of cereal, bowls, and spoons are stored in adjacent kitchen cabinets/drawers – spatial locality
* You repeatedly check your phone for new messages – temporal locality
* A variable used in a for loop – temporal locality
* Variables used in different functions – neither
* A function's parameters, which are each used once within the function – spatial locality

* Gates for flights on the same airline are located in the same airport terminal/concourse
* A grocery list is arranged in alphabetical order
* Clothes in a closet are grouped into outfits, with a shirt and a pair of pants stored next to each other
* Boxes of cereal, bowls, and spoons are stored in adjacent kitchen cabinets/drawers
* You repeatedly check your phone for new messages
* A variable used in a for loop
* Variables used in different functions
* A function's parameters, which are each used once within the function

Q7: _For each of the following instances of caching, indicate whether the caching is motivated by temporal or spatial locality._

* A CPU caches the first 32 instructions of a function when the function is called – spatial
* A CPU caches all of the instructions for a frequently called function – temporal
* A web browser caches the Moodle pages for your courses, which you view multiple times per week – temporal
* A content distribution network (CDN) caches a video that has gone viral – temporal
* A content distribution network (CDN) caches "recommended videos" related to a video – spatial

* A CPU caches the first 32 instructions of a function when the function is called
* A CPU caches all of the instructions for a frequently called function
* A web browser caches the Moodle pages for your courses, which you view multiple times per week
* A content distribution network (CDN) caches a video that has gone viral
* A content distribution network (CDN) caches "recommended videos" related to a video